# プレーンなTensorFlowを使ったDNNの訓練

In [62]:
import tensorflow as tf
import numpy as np
n_inputs =28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [63]:
X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X")
y = tf.placeholder(tf.int64, shape = (None), name="y")

In [64]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])#サイズを調べている[0]だとインスタンス数
        stddev = 2/np.sqrt(n_inputs + n_neurons) # 切断正規分布
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf. matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [65]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    
    logits = neuron_layer(hidden2, n_outputs, name="outputs") # ソフトマックス活性化関数に通す前の出力

今回は関数を自作したが、このようにdense()を使ってできる。
```
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
```

In [66]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits=logits)
    #各インスタンスの交差エントロピーが収められた一次元のテンソル
    loss = tf.reduce_mean(xentropy, name="loss")

In [67]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [68]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)#論理値を集めた一次元テンソルを返す。
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))#それを浮動小数点にキャストして平均を計算する

In [69]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()#var_list={"W":W, "b":b, "W_1":W_1, "b_1":b_1, "W_2":W_2, "b_2":b_2}はできなかった

### 実行フェーズ

In [76]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

#ここではテスト用にしか使っていない
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

acc_summary = tf.summary.scalar('Accuracy', accuracy)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [71]:
n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict = {X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict = {X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict = {X: mnist.validation.images, y: mnist.validation.labels})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Train accuracy: 0.86 Val accuracy: 0.9138
1 Train accuracy: 0.92 Val accuracy: 0.928
2 Train accuracy: 0.94 Val accuracy: 0.9404
3 Train accuracy: 0.94 Val accuracy: 0.9454
4 Train accuracy: 0.98 Val accuracy: 0.951
5 Train accuracy: 0.96 Val accuracy: 0.9532
6 Train accuracy: 0.98 Val accuracy: 0.9574
7 Train accuracy: 0.96 Val accuracy: 0.9618
8 Train accuracy: 0.92 Val accuracy: 0.9622
9 Train accuracy: 1.0 Val accuracy: 0.963
10 Train accuracy: 0.96 Val accuracy: 0.9642
11 Train accuracy: 1.0 Val accuracy: 0.9664
12 Train accuracy: 0.98 Val accuracy: 0.966
13 Train accuracy: 1.0 Val accuracy: 0.968
14 Train accuracy: 0.98 Val accuracy: 0.9694
15 Train accuracy: 0.98 Val accuracy: 0.9702
16 Train accuracy: 0.98 Val accuracy: 0.9712
17 Train accuracy: 0.98 Val accuracy: 0.9708
18 Train accuracy: 1.0 Val accuracy: 0.9726
19 Train accuracy: 1.0 Val accuracy: 0.9732
20 Train accuracy: 0.98 Val accuracy: 0.9724
21 Train accuracy: 0.98 Val accuracy: 0.9734
22 Train accuracy: 1.0 Val acc

In [81]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    X_new_scaled =X_test[:20]
    Z = logits.eval(feed_dict={X:X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [85]:
print(y_pred)

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


In [84]:
print(y_test[:20])

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
